In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix


In [2]:
class SpotifyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class SpotifyRankPredictor(nn.Module):
    def __init__(self, num_categories):
        super(SpotifyRankPredictor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.2),

            nn.Linear(32, num_categories)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.layers(x)
        return self.softmax(x)

def augment_features(features, num_samples_needed):
    augmented_data = []
    feature_names = ['Danceability', 'Energy', 'Loudness', 'Speechiness',
                     'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)']

    # 노이즈 범위 조정 - 더 섬세한 변화를 위해 값들을 약간 줄임
    noise_ranges = {
        'Danceability': 0.03,
        'Energy': 0.03,
        'Loudness': 0.8,
        'Speechiness': 0.015,
        'Acousticness': 0.03,
        'Liveness': 0.03,
        'Tempo': 2.0,
        'Duration (ms)': 0.03
    }

    for _ in range(num_samples_needed):
        base_sample = features[np.random.randint(len(features))]
        new_features = []

        for feat_idx, feat_name in enumerate(feature_names):
            feature = np.abs(base_sample[feat_idx])

            if feat_name == 'Loudness':
                noise = np.random.normal(0, np.abs(noise_ranges['Loudness']))
            elif feat_name == 'Tempo':
                noise = np.random.normal(0, np.abs(noise_ranges['Tempo']))
            elif feat_name == 'Duration (ms)':
                noise = np.random.normal(0, np.abs(feature * noise_ranges['Duration (ms)']))
            else:
                noise = np.random.normal(0, np.abs(feature * noise_ranges[feat_name]))

            new_value = feature + noise

            if feat_name == 'Loudness':
                new_value = np.clip(new_value, -60, 0)
            elif feat_name == 'Duration (ms)':
                new_value = max(1000, new_value)
            else:
                new_value = np.clip(new_value, 0, 1)

            new_features.append(new_value)

        augmented_data.append(new_features)

    return np.array(augmented_data)

def calculate_class_weights(categories):
    class_counts = np.bincount(categories)
    total_samples = np.sum(class_counts)
    class_weights = total_samples / (len(class_counts) * class_counts)
    return torch.FloatTensor(class_weights)

def preprocess_data(df):
    X = df[['Danceability', 'Energy', 'Loudness', 'Speechiness',
            'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)']].values

    def rank_to_category(rank):
        if rank <= 10:
            return 0
        elif rank <= 30:
            return 1
        elif rank <= 50:
            return 2
        elif rank <= 100:
            return 3
        else:
            return 4

    ranks = df['Highest Charting Position'].values
    categories = np.array([rank_to_category(rank) for rank in ranks])

    # 원-핫 인코딩
    num_categories = 5
    y_encoded = np.eye(num_categories)[categories]

    return X, y_encoded, num_categories

def augment_and_balance_training_data(X, y, scaler):
    # one-hot 인코딩된 y를 다시 카테고리로 변환
    categories = np.argmax(y, axis=1)

    # 각 카테고리별 샘플 수 계산
    category_counts = np.bincount(categories)
    print("Original category counts in train set:", category_counts)

    # 목표 샘플 수 설정 (가장 많은 클래스의 90%로 조정)
    target_samples = int(max(category_counts) * 0.9)
    print("Target samples per category:", target_samples)

    processed_features = []
    processed_categories = []

    for category in range(5):
        category_mask = categories == category
        category_features = X[category_mask]
        current_samples = len(category_features)

        if current_samples < target_samples:
            num_samples_needed = target_samples - current_samples
            augmented = augment_features(category_features, num_samples_needed)
            processed_features.append(np.vstack([category_features, augmented]))
            processed_categories.extend([category] * target_samples)
        else:
            processed_features.append(category_features)
            processed_categories.extend([category] * current_samples)

    X_combined = np.vstack(processed_features)
    categories_combined = np.array(processed_categories)

    # 스케일링된 특성
    X_scaled = scaler.transform(X_combined)

    # 원-핫 인코딩
    y_combined = np.eye(5)[categories_combined]

    print("Final category counts in balanced train set:", np.bincount(categories_combined))

    # 클래스 가중치 계산
    class_weights = calculate_class_weights(categories_combined)

    return X_scaled, y_combined, class_weights

def train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs=150, patience=15):  # 에포크 수와 patience 증가
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = criterion.to(device)  # 가중치가 적용된 criterion을 device로 이동

    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []
    best_val_accuracy = 0  # 최고 검증 정확도 추적

    for epoch in range(num_epochs):
        # 훈련 단계
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()

            # Gradient Clipping 추가
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            train_total += y_batch.size(0)
            train_correct += (predicted == actual).sum().item()

        train_loss /= len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        train_losses.append(train_loss)

        # 검증 단계
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                _, actual = torch.max(y_batch.data, 1)
                val_total += y_batch.size(0)
                val_correct += (predicted == actual).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        val_losses.append(val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        # 모델 저장 조건을 검증 정확도 기준으로 변경
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': val_accuracy,
            }, 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping triggered! Best validation accuracy: {best_val_accuracy:.2f}%')
                break

    return train_losses, val_losses

def main():
    # 1. 데이터 로드 및 기본 전처리
    df = pd.read_csv('spotify_dataset.csv')
    X, y_encoded, num_categories = preprocess_data(df)

    # 2. 데이터 분할 (증강 전)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42,
        stratify=y_encoded.argmax(axis=1)
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42,
        stratify=y_temp.argmax(axis=1)
    )

    # 3. StandardScaler 학습 (train set으로만)
    scaler = StandardScaler()
    scaler.fit(X_train)

    # 4. train 데이터에만 증강 및 밸런싱 적용
    X_train_processed, y_train_processed, class_weights = augment_and_balance_training_data(
        X_train, y_train, scaler
    )

    # 5. validation과 test set은 스케일링만 적용
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # 6. 데이터셋 생성
    train_dataset = SpotifyDataset(X_train_processed, y_train_processed)
    val_dataset = SpotifyDataset(X_val_scaled, y_val)
    test_dataset = SpotifyDataset(X_test_scaled, y_test)

    batch_size = 64
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = SpotifyRankPredictor(num_categories)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, verbose=True
    )

    train_losses, val_losses = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=150,
        patience=15
    )

    # 최고 성능 모델 불러오기
    checkpoint = torch.load('best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # 테스트 세트에서 평가
    test_loss = 0
    correct = 0
    total = 0
    y_pred = []
    y_true = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            total += y_batch.size(0)
            correct += (predicted == actual).sum().item()
            y_pred.extend(predicted.numpy())
            y_true.extend(actual.numpy())

    accuracy = 100 * correct / total
    print(f'\nTest Accuracy: {accuracy:.2f}%')

    conf_matrix = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)
    print("Class weights:", class_weights)

if __name__ == "__main__":
    main()

Original category counts in train set: [202 260 188 452 650]
Target samples per category: 585
Final category counts in balanced train set: [585 585 585 585 650]


C:\anaconda3\envs\DeepL_yonsei\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/150], Train Loss: 1.5674, Train Acc: 30.54%, Val Loss: 1.5210, Val Acc: 36.53%
Epoch [2/150], Train Loss: 1.5368, Train Acc: 35.15%, Val Loss: 1.5071, Val Acc: 33.79%
Epoch [3/150], Train Loss: 1.5261, Train Acc: 35.45%, Val Loss: 1.5032, Val Acc: 33.33%
Epoch [4/150], Train Loss: 1.5209, Train Acc: 36.76%, Val Loss: 1.5028, Val Acc: 33.79%
Epoch [5/150], Train Loss: 1.5175, Train Acc: 37.16%, Val Loss: 1.4985, Val Acc: 35.62%
Epoch [6/150], Train Loss: 1.5145, Train Acc: 37.76%, Val Loss: 1.4940, Val Acc: 36.53%
Epoch [7/150], Train Loss: 1.5082, Train Acc: 38.29%, Val Loss: 1.4947, Val Acc: 34.25%
Epoch [8/150], Train Loss: 1.5077, Train Acc: 38.26%, Val Loss: 1.4944, Val Acc: 33.79%
Epoch [9/150], Train Loss: 1.5050, Train Acc: 37.86%, Val Loss: 1.4896, Val Acc: 34.70%
Epoch [10/150], Train Loss: 1.5037, Train Acc: 39.16%, Val Loss: 1.4883, Val Acc: 33.79%
Epoch [11/150], Train Loss: 1.4980, Train Acc: 39.20%, Val Loss: 1.4847, Val Acc: 35.62%
Epoch [12/150], Train Loss: 1.

C:\Users\이강민\AppData\Local\Temp\ipykernel_18264\2332630081.py:303: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_model.pth')
